In [1]:
import pandas as pd
from sklearn.cross_validation import train_test_split
import numpy as np
from matplotlib import pyplot as plt
import xgboost as xgb
import random
% matplotlib inline

from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

In [ ]:
BP = './f1/'
X_test = pd.read_csv(BP + 'X_test.csv').drop(['Id', 'qid'], axis=1)
X_train = pd.read_csv(BP + 'X_train.csv').drop(['Id', 'qid'], axis=1)
X_val = pd.read_csv(BP + 'X_val.csv').drop(['Id', 'qid'], axis=1)

y_test = pd.read_csv(BP + 'y_test.csv').drop('Id', axis=1)
y_train = pd.read_csv(BP + 'y_train.csv').drop('Id', axis=1)
y_val = pd.read_csv(BP + 'y_val.csv').drop('Id', axis=1)

In [ ]:
BP = './f2/'
X_test2 = pd.read_csv(BP + 'X_test.csv').drop(['Id', 'qid'], axis=1)
X_train2 = pd.read_csv(BP + 'X_train.csv').drop(['Id', 'qid'], axis=1)
X_val2 = pd.read_csv(BP + 'X_val.csv').drop(['Id', 'qid'], axis=1)

y_test2 = pd.read_csv(BP + 'y_test.csv').drop('Id', axis=1)
y_train2 = pd.read_csv(BP + 'y_train.csv').drop('Id', axis=1)
y_val2 = pd.read_csv(BP + 'y_val.csv').drop('Id', axis=1)

In [ ]:
model = xgb.XGBClassifier(objective='rank:ndcg')
model.fit(X_train, y_train, eval_metric='ndcg')
predictions = model.predict(X_val)
print "ndcg : %.4g" % ndcg_score(y_val, predictions)

In [ ]:
xgtrain = xgb.DMatrix(X_train, label=y_train)
xgtest = xgb.DMatrix(X_val, label=y_val)

In [ ]:
param = {}
param['silent'] = 0
param['booster'] = 'gbtree'
param['objective'] = 'rank:pairwise'
param["eval_metric"] = 'ndcg'
param["n_estimators"] = 1000
evals = [(xgtrain, 'train'), (xgtest, 'val')]
dicteval = {(xgtrain, 'train'): 'ndcg', (xgtest, 'val') :'ndcg'}
param['eta'] = 0.6
n_iter = 1000
bst = xgb.train(param, xgtrain, n_iter, evals = evals, verbose_eval=n_iter-1, evals_result=dicteval)

In [ ]:
epochs = len(dicteval['train'][''])
x_axis = range(0, epochs)
# plot log loss
fig, ax = plt.subplots()
ax.plot(x_axis, dicteval['train'][''], label='Train')
ax.plot(x_axis, dicteval['val'][''], label='Test')
ax.legend()
plt.ylabel('NDCG')
plt.title('XGBoost NDCG')
plt.show()

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import make_scorer


def dcg_score(y_true, y_score, k=5):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    gain = 2 ** y_true - 1

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)


def ndcg_score(ground_truth, predictions, k=5):
    lb = LabelBinarizer()
    T = lb.fit_transform(ground_truth)

    scores = []

    # Iterate over each y_true and compute the DCG score
    for y_true, y_score in zip(T, predictions):
        actual = dcg_score(y_true, y_score, k)
        best = dcg_score(y_true, y_true, k)
        score = float(actual) / float(best)
        scores.append(score)

    return np.mean(scores)

# NDCG Scorer function
ndcg_scorer = make_scorer(ndcg_score, needs_proba=True)


In [3]:
def cv_eta(X_train, y_train, X_val, y_val):

    xgtrain = xgb.DMatrix(X_train, label=y_train)
    xgtest = xgb.DMatrix(X_val, label=y_val)
    
    param = {}
    param['silent'] = 0
    param['booster'] = 'gbtree'
    param['objective'] = 'rank:pairwise'
    param["eval_metric"] = 'ndcg'
    param["n_estimators"] = 1000
    num_round = 150
    results = []

    for eta in np.linspace(0.1, 2, 20):
        param['eta'] = eta
        cv = xgb.cv(param, xgtrain, num_round, nfold=5, metrics={'ndcg'}, seed = 0)
        results.append(cv['test--mean'][99])
        print "learning_rate: ", eta, "ndcg: ", cv['test--mean'][99]
    return results

In [ ]:
#original_pool
size = X_train.shape[0]
res = cv_eta(X_train[:size], y_train[:size], X_val, y_val)

In [ ]:
#original_pool_2
size = X_train.shape[0] / 2
res2 = cv_eta(X_train[:size], y_train[:size], X_val, y_val)

In [ ]:
#original_pool_5
size = X_train.shape[0] / 5
res5 = cv_eta(X_train[:size], y_train[:size], X_val, y_val)

In [ ]:
#original_pool_10
size = X_train.shape[0] / 10
res10 = cv_eta(X_train[:size], y_train[:size], X_val, y_val)

In [4]:
def noisy_pool(y):
    size = y.shape[0]
    set_of_values = list(set(y))
    distr = np.zeros(size)
    for i in range(size / 100):
        distr[i] = 1
    noisy_y = [random.choice(set_of_values) if random.choice(distr) else y[i] for i in range(size)]
    return pd.DataFrame(noisy_y)

y_train_n = noisy_pool(y_train["label"])

In [ ]:
#noisy_pool
size = X_train.shape[0]
res_n = cv_eta(X_train[:size], y_train[:size], X_val, y_val)

In [ ]:
#noisy_pool2
size = X_train.shape[0] / 2
res_n2 = cv_eta(X_train[:size], y_train[:size], X_val, y_val)

In [ ]:
#noisy_pool5
size = X_train.shape[0] / 5
res_n5 = cv_eta(X_train[:size], y_train[:size], X_val, y_val)

In [ ]:
#noisy_pool10
size = X_train.shape[0] / 10
res_n10 = cv_eta(X_train[:size], y_train[:size], X_val, y_val)